# Transformer

Transformer의 핵심 구조인 Multi-head Attention을 구현하는 실습입니다.
1. Multi-head attention 및 self-attention 구현.
2. 각 과정에서 일어나는 연산과 input/output 형태 이해.

### 필요 패키지 import

In [1]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

## Req. 2-1 Multi-head self-attention 구조 익히기

### **데이터 전처리**
vocab_size 100인 가상의 시퀀스 데이터를 생성합니다. 

각 데이터에 할당된 숫자는 tokenizing과 정수화가 이뤄진 형태입니다.

In [2]:
pad_id = 0
vocab_size = 100

data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

In [3]:
# 길이 맞춰주기 위해 패딩합니다.
def padding(data):
  max_len = len(max(data, key=len))
  print(f"Maximum sequence length: {max_len}")

  for i, seq in enumerate(tqdm(data)):
    if len(seq) < max_len:
      data[i] = seq + [pad_id] * (max_len - len(seq))

  return data, max_len

In [4]:
data, max_len = padding(data)

Maximum sequence length: 20


100%|██████████| 10/10 [00:00<00:00, 17593.56it/s]


In [5]:
data

[[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0],
 [60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [77,
  65,
  51,
  77,
  19,
  15,
  35,
  19,
  23,
  97,
  50,
  46,
  53,
  42,
  45,
  91,
  66,
  3,
  43,
  10],
 [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0],
 [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]]

### Hyperparameter 세팅 및 embedding

In [6]:
d_model = 512  # model의 hidden size
num_heads = 8  # head의 개수

# d_model이 입력을 projection 시킬 임베딩 space의 차원이므로, num_heads로 나누어 떨어져야 한다.

In [7]:
embedding = nn.Embedding(vocab_size, d_model)

# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_emb = embedding(batch)  # (B, L, d_model)

In [8]:
print(batch_emb)
print(batch_emb.shape)

tensor([[[ 0.8299, -1.0274,  0.2678,  ...,  0.9656, -0.8258,  2.0953],
         [-0.5363, -1.5044, -0.1813,  ...,  0.9791, -0.8086,  1.3326],
         [-0.8490, -1.3976, -0.0542,  ...,  1.3797,  0.6005, -0.8897],
         ...,
         [ 0.7753,  0.2142,  0.0456,  ..., -0.2554,  1.1011,  0.4315],
         [ 0.7753,  0.2142,  0.0456,  ..., -0.2554,  1.1011,  0.4315],
         [ 0.7753,  0.2142,  0.0456,  ..., -0.2554,  1.1011,  0.4315]],

        [[-0.3280,  0.6802,  0.5387,  ...,  0.3527, -0.2873,  1.3021],
         [ 0.1888, -0.9981, -0.4318,  ...,  0.8275,  0.1479,  0.2592],
         [-0.3581, -0.3562,  0.8462,  ...,  0.2870,  0.6184,  0.1386],
         ...,
         [ 0.7753,  0.2142,  0.0456,  ..., -0.2554,  1.1011,  0.4315],
         [ 0.7753,  0.2142,  0.0456,  ..., -0.2554,  1.1011,  0.4315],
         [ 0.7753,  0.2142,  0.0456,  ..., -0.2554,  1.1011,  0.4315]],

        [[-0.1300, -0.2602, -0.4899,  ...,  0.0735,  2.4445,  0.6219],
         [ 0.8364, -0.8342,  0.3948,  ...,  0

### Linear projection & 여러 head로 나누기

Multi-head attention 내에서 쓰이는 linear projection matrix들을 정의합니다.

In [9]:
w_q = nn.Linear(d_model, d_model)
w_k = nn.Linear(d_model, d_model)
w_v = nn.Linear(d_model, d_model)

In [10]:
w_0 = nn.Linear(d_model, d_model)

In [11]:
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 512])
torch.Size([10, 20, 512])
torch.Size([10, 20, 512])


Q, k, v를 `num_head`개의 차원 분할된 여러 vector로 만듭니다.

- 이론적으로는 multi-head attention을 수행하면 input을 각각 다른 head 개수만큼의 Wq, Wk, Wv로 linear transformation 해서 각각 여러번의 attention 수행한 후 concat 한 후 linear transformation 수행해준다
- 구현에서는 Wq, Wk, Wv 한 개씩
- 실제 `attention is all you need` 논문의 구현 예시는 Query vector 한개를 dim으로 쪼개서 진행한다

In [12]:
batch_size = q.shape[0]
d_k = d_model // num_heads

# num_heads * d_k로 쪼갠다
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])
torch.Size([10, 20, 8, 64])


In [13]:
# num_heads를 밖으로 뺌으로써
# 각 head가 (L, d_k) 만큼의 matrix를 가지고 self-attention 수행

q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

print(q.shape)
print(k.shape)
print(v.shape)

torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])
torch.Size([10, 8, 20, 64])


### Scaled dot-product self-attention 구현

각 head에서 실행되는 self-attetion 과정입니다.

In [14]:
# shape - (L, L)
# 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
# softmax - row-wise이기 때문에 dim은 -1
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

print(attn_dists)
print(attn_dists.shape)

tensor([[[[0.0478, 0.0460, 0.0667,  ..., 0.0318, 0.0318, 0.0318],
          [0.0934, 0.0463, 0.0583,  ..., 0.0451, 0.0451, 0.0451],
          [0.0351, 0.0502, 0.0856,  ..., 0.0629, 0.0629, 0.0629],
          ...,
          [0.0384, 0.0524, 0.0689,  ..., 0.0369, 0.0369, 0.0369],
          [0.0384, 0.0524, 0.0689,  ..., 0.0369, 0.0369, 0.0369],
          [0.0384, 0.0524, 0.0689,  ..., 0.0369, 0.0369, 0.0369]],

         [[0.0374, 0.0378, 0.0504,  ..., 0.0362, 0.0362, 0.0362],
          [0.0312, 0.0799, 0.0456,  ..., 0.0572, 0.0572, 0.0572],
          [0.0602, 0.0407, 0.0275,  ..., 0.0571, 0.0571, 0.0571],
          ...,
          [0.0607, 0.0558, 0.0573,  ..., 0.0438, 0.0438, 0.0438],
          [0.0607, 0.0558, 0.0573,  ..., 0.0438, 0.0438, 0.0438],
          [0.0607, 0.0558, 0.0573,  ..., 0.0438, 0.0438, 0.0438]],

         [[0.0247, 0.0427, 0.0314,  ..., 0.0654, 0.0654, 0.0654],
          [0.0310, 0.0879, 0.0359,  ..., 0.0422, 0.0422, 0.0422],
          [0.0539, 0.0492, 0.0637,  ..., 0

In [15]:
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

print(attn_values.shape)

torch.Size([10, 8, 20, 64])


### 각 head의 결과물 병합

각 head의 결과물을 concat하고 동일 차원으로 linear projection합니다.

In [16]:
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)
attn_values = attn_values.contiguous().view(batch_size, -1, d_model)  # (B, L, d_model)

print(attn_values.shape)

torch.Size([10, 20, 512])


In [17]:
# w_0 : (d_model, d_model)
# 서로 다른 의미로 foucsing 된 각 head의 self-attention 정보들을 합쳐주는 역할 수행
outputs = w_0(attn_values)

print(outputs)
print(outputs.shape)

tensor([[[ 0.0340,  0.1785,  0.1140,  ..., -0.2309,  0.0606, -0.0093],
         [-0.1011,  0.1640,  0.1647,  ..., -0.2290,  0.0757, -0.0131],
         [-0.0088,  0.1614,  0.1052,  ..., -0.1820,  0.0577,  0.0167],
         ...,
         [-0.0282,  0.1478,  0.1180,  ..., -0.2435,  0.0289, -0.0318],
         [-0.0282,  0.1478,  0.1180,  ..., -0.2435,  0.0289, -0.0318],
         [-0.0282,  0.1478,  0.1180,  ..., -0.2435,  0.0289, -0.0318]],

        [[-0.0935,  0.3263,  0.4433,  ..., -0.1985,  0.0151,  0.0284],
         [-0.1923,  0.2931,  0.4930,  ..., -0.2123,  0.0227,  0.0939],
         [-0.0979,  0.3147,  0.4543,  ..., -0.1673,  0.0500,  0.0169],
         ...,
         [-0.1700,  0.3166,  0.4764,  ..., -0.1604,  0.0295,  0.0503],
         [-0.1700,  0.3166,  0.4764,  ..., -0.1604,  0.0295,  0.0503],
         [-0.1700,  0.3166,  0.4764,  ..., -0.1604,  0.0295,  0.0503]],

        [[-0.0119,  0.3691,  0.2650,  ..., -0.0173,  0.1910, -0.0545],
         [-0.0683,  0.3236,  0.2740,  ..., -0

## Req. 2-2 Multi-head self-attention 모듈 클래스 구현

위의 과정을 모두 합쳐 하나의 Multi-head attention 모듈 class를 구현하겠습니다.

아래 코드의 TODO 부분을 채워주세요.

In [18]:
class MultiheadAttention(nn.Module):
  def __init__(self):
    super(MultiheadAttention, self).__init__()

    # Q, K, V learnable matrices
    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)

    # Linear projection for concatenated outputs
    self.w_0 = nn.Linear(d_model, d_model)

  # scaled-dot product attention
  def self_attention(self, q, k, v):
    attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)  # (B, num_heads, L, L)
    attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

    attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

    return attn_values

  def forward(self, q, k, v):
    batch_size = q.shape[0]

    # linear projection
    ################################################################################
    # TODO 1: Implement the forward pass for linear projection.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # head만큼 쪼개준다
    ################################################################################
    # TODO 2: Implement the forward pass for split head.                #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # 각 head가 (L, d_k)의 matrix를 담당하도록 만든다
    q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
    k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
    v = v.transpose(1, 2)  # (B, num_heads, L, d_k)

    attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)
    attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, d_model)  # (B, L, num_heads, d_k) => (B, L, d_model)

    return self.w_0(attn_values)

In [19]:
multihead_attn = MultiheadAttention()

outputs = multihead_attn(batch_emb, batch_emb, batch_emb)  # (B, L, d_model)

In [20]:
print(outputs)
print(outputs.shape)  # (batch_size, length, d_model)

tensor([[[-5.8577e-02,  1.9090e-01,  3.0195e-02,  ...,  9.1379e-02,
          -1.1316e-01, -2.2785e-02],
         [ 1.6478e-01,  1.0062e-01,  1.3772e-01,  ...,  1.1550e-01,
          -2.2411e-01,  2.7540e-01],
         [ 4.3012e-02, -4.0370e-02, -1.1256e-01,  ..., -4.4907e-02,
           7.4560e-02, -1.4822e-01],
         ...,
         [-1.9461e-01, -6.9398e-02,  2.2076e-01,  ...,  2.7266e-01,
           1.5912e-01,  1.3727e-01],
         [-1.9461e-01, -6.9398e-02,  2.2076e-01,  ...,  2.7266e-01,
           1.5912e-01,  1.3727e-01],
         [-1.9461e-01, -6.9398e-02,  2.2076e-01,  ...,  2.7266e-01,
           1.5912e-01,  1.3727e-01]],

        [[ 5.6246e-02,  2.0459e-02,  5.8797e-02,  ..., -1.1778e-01,
          -5.1849e-02,  6.4371e-02],
         [-9.1925e-02, -1.9409e-01, -1.1947e-01,  ...,  8.2040e-02,
          -2.3640e-03, -1.7494e-01],
         [ 5.3210e-02,  7.9480e-02, -4.3399e-02,  ...,  7.6058e-02,
           4.4118e-02,  1.5110e-02],
         ...,
         [-5.1321e-01, -3